# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

In [26]:
# --- Step 1: Prep + join states onto citations ---
# Goal: create the intermediate table:
#   (CITING, CITED, CITED_STATE, CITING_STATE)
# We drop null/blank POSTATE up front so comparisons later are clean.
# Left joins are intentional: not every cited/citing patent has state info.

from pyspark.sql.functions import col

# Clean mapping: patent -> state
pat_states_clean = (
    patents
    .select("PATENT", "POSTATE")
    .filter(col("POSTATE").isNotNull() & (col("POSTATE") != ""))
    .cache()
)

# Attach cited patent state
cited_state = (
    citations
    .join(
        pat_states_clean.withColumnRenamed("PATENT", "CITED"),
        on="CITED",
        how="left"
    )
    .select("CITING", "CITED", col("POSTATE").alias("CITED_STATE"))
    .cache()
)

# Attach citing patent state (now we have both sides)
both_states = (
    cited_state
    .join(
        pat_states_clean.withColumnRenamed("PATENT", "CITING"),
        on="CITING",
        how="left"
    )
    .select("CITING", "CITED", "CITED_STATE", col("POSTATE").alias("CITING_STATE"))
    .cache()
)

# Quick peek just to confirm columns look right (nulls are expected)
both_states.show(10, truncate=False)

+-------+-------+-----------+------------+
|CITING |CITED  |CITED_STATE|CITING_STATE|
+-------+-------+-----------+------------+
|3858258|1540798|NULL       |CA          |
|3858258|1331793|NULL       |CA          |
|3858527|3638586|CA         |NULL        |
|3858527|924225 |NULL       |NULL        |
|3858527|2444326|NULL       |NULL        |
|3858527|3699902|OH         |NULL        |
|3858527|2967080|NULL       |NULL        |
|3858527|3602157|TX         |NULL        |
|3858527|2705120|NULL       |NULL        |
|3858560|957631 |NULL       |IN          |
+-------+-------+-----------+------------+
only showing top 10 rows



### Step 1: Prepare patent state data and attach it to citations

In this step, I prepare the data needed to identify same-state patent citations.  
I begin by creating a clean mapping from patent number to state (`PATENT → POSTATE`), keeping only patents with valid state information. This helps avoid issues later when comparing states or counting citations.

Next, I join this state information onto the citation dataset twice. The first join attaches the state of the **cited** patent, and the second join attaches the state of the **citing** patent. Left joins are used intentionally because not all patents appear in both datasets and missing state values are expected.

The result of this step is an intermediate table containing the citing patent, the cited patent, and the state associated with each. This table is used directly in the next step to identify and count same-state citations.

In [27]:
# --- Step 2: Count same-state citations + join back onto patents + top 10 ---
# Same-state citation = both states exist AND match.
# Then count per CITING patent, attach as SAME_STATE (missing -> 0),
# and print the top 10 patents by SAME_STATE (descending).

from pyspark.sql.functions import count, lit, coalesce

same_state_counts = (
    both_states
    .filter(col("CITED_STATE").isNotNull() & col("CITING_STATE").isNotNull())
    .filter(col("CITED_STATE") == col("CITING_STATE"))
    .groupBy("CITING")
    .agg(count(lit(1)).alias("SAME_STATE"))
    .cache()
)

patents_aug = (
    patents
    .join(
        same_state_counts,
        patents["PATENT"].cast("int") == same_state_counts["CITING"],
        how="left"
    )
    .drop("CITING")
    .withColumn("SAME_STATE", coalesce(col("SAME_STATE"), lit(0)))
    .cache()
)

top10 = (
    patents_aug
    .filter(col("POSTATE").isNotNull() & (col("POSTATE") != ""))  # keep US-state patents
    .select("PATENT", "POSTATE", "SAME_STATE")
    .orderBy(col("SAME_STATE").desc(), col("PATENT").asc())
    .limit(10)
)

top10.show(truncate=False)

+-------+-------+----------+
|PATENT |POSTATE|SAME_STATE|
+-------+-------+----------+
|5959466|CA     |125       |
|5983822|TX     |103       |
|6008204|CA     |100       |
|5952345|CA     |98        |
|5958954|CA     |96        |
|5998655|CA     |96        |
|5936426|CA     |94        |
|5739256|CA     |90        |
|5913855|CA     |90        |
|5925042|CA     |90        |
+-------+-------+----------+



### Step 2: Count same-state citations and identify the top patents

In this step, I compute the main value required for the assignment: the number of same-state citations for each patent. A same-state citation is defined as a citation where both the citing patent and the cited patent have state information and originate from the same state.

Using the intermediate table from the previous step, I first filter out any rows where state information is missing. I then keep only the cases where the cited and citing states match. These filtered rows are grouped by the citing patent and counted to determine how many same-state citations each patent has.

This count is joined back onto the original patent dataset as a new column called `SAME_STATE`. Patents without any same-state citations are assigned a value of zero. Finally, the results are sorted in descending order to identify the top ten patents with the highest number of same-state citations.

In [28]:
# --- Reference-style output ---
# Show the full patent record with SAME_STATE appended at the end.
# This matches the format shown in the lab README screenshot
# (all original patent columns + SAME_STATE, sorted by SAME_STATE).

patents_aug_ref = patents_aug.select(*patents.columns, "SAME_STATE")

patents_aug_ref \
    .orderBy(col("SAME_STATE").desc()) \
    .show(10, truncate=False)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|PATENT |GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466|1999 |14515|1997   |US     |CA     |5310    |2      |NULL  |326   |4  |46    |159  |0       |1.0     |NULL   |0.6186  |NULL    |4.8868  |0.0455  |0.044   |NULL    |NULL    |125       |
|5983822|1999 |14564|1998   |US     |TX     |569900  |2      |NULL  |114   |5  |55    |200  |0       |0.995   |NULL   |0.7201  |NULL    |12.45   |0.0     |0.0     |NULL    |NULL    |103       |
|6008204|1999 |14606|1998   |U

### Final output (DataFrame solution)

My final solution uses the PySpark **DataFrame** method. I attach state information to citations (for both the cited and citing patents), count how many citations stay within the same state, and then add that count back onto the patent table as `SAME_STATE` (missing values become 0). This final output prints the top 10 patents with the highest `SAME_STATE` values in the same wide-table format as the reference.